<a href="https://colab.research.google.com/github/rgnemasters/nfl-combine-draft-2022/blob/main/Combine_ML_Modelling_Steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Drive, Import Libraries, Load Files

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [48]:
#Dataframe and viz libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Models and pipelines
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm.sklearn import LGBMRegressor

#Transformers and imputers
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer

#Metrics
from sklearn.metrics import r2_score, mean_squared_error

In [23]:
df = pd.read_excel('/content/drive/MyDrive/Datasets/combine_table.xlsx')
qb_df = pd.read_excel('/content/drive/MyDrive/Datasets/combine_qb_table.xlsx')
wr_df = pd.read_excel('/content/drive/MyDrive/Datasets/combine_wr_table.xlsx')
rb_df = pd.read_excel('/content/drive/MyDrive/Datasets/combine_rb_table.xlsx')

#Review datasets

In [24]:
print(df.info())
display(df.head())
round(df.isna().sum() / len(df), 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7327 entries, 0 to 7326
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   7327 non-null   int64  
 1   Player       7327 non-null   object 
 2   Pos          7327 non-null   object 
 3   School       7327 non-null   object 
 4   Ht           7327 non-null   int64  
 5   Wt           7327 non-null   int64  
 6   40yd         6956 non-null   float64
 7   Vertical     5696 non-null   float64
 8   Bench        5009 non-null   float64
 9   Broad Jump   5632 non-null   float64
 10  3Cone        4692 non-null   float64
 11  Shuttle      4787 non-null   float64
 12  Year         7327 non-null   int64  
 13  team         7327 non-null   object 
 14  round        7327 non-null   int64  
 15  pick number  7327 non-null   int64  
dtypes: float64(6), int64(6), object(4)
memory usage: 916.0+ KB
None


,Unnamed: 0,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Year,team,round,pick number
0,0,John Abraham,OLB,South Carolina,76,252,4.55,NaN,NaN,NaN,NaN,NaN,2000,New York Jets,1,13
1,1,Shaun Alexander,RB,Alabama,72,218,4.58,NaN,NaN,NaN,NaN,NaN,2000,Seattle Seahawks,1,19
2,2,Darnell Alford,OT,Boston Col.,76,334,5.56,25.0,23.0,94.0,8.48,4.98,2000,Kansas City Chiefs,6,188
3,3,Kyle Allamon,TE,Texas Tech,74,253,4.97,29.0,NaN,104.0,7.29,4.49,2000,Undrafted,8,260
4,4,Rashard Anderson,CB,Jackson State,74,206,4.55,34.0,NaN,123.0,7.18,4.15,2000,Carolina Panthers,1,23


Unnamed: 0     0.00
Player         0.00
Pos            0.00
School         0.00
Ht             0.00
Wt             0.00
40yd           0.05
Vertical       0.22
Bench          0.32
Broad Jump     0.23
3Cone          0.36
Shuttle        0.35
Year           0.00
team           0.00
round          0.00
pick number    0.00
dtype: float64

In [25]:
print(qb_df.info())
display(qb_df.head())
round(qb_df.isna().sum() / len(qb_df), 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        401 non-null    int64  
 1   Player            401 non-null    object 
 2   Pos               401 non-null    object 
 3   School            401 non-null    object 
 4   Ht                401 non-null    int64  
 5   Wt                401 non-null    int64  
 6   40yd              384 non-null    float64
 7   Vertical          329 non-null    float64
 8   Broad Jump        331 non-null    float64
 9   3Cone             310 non-null    float64
 10  Shuttle           314 non-null    float64
 11  Year              401 non-null    int64  
 12  team              401 non-null    object 
 13  round             401 non-null    int64  
 14  pick number       401 non-null    int64  
 15  Y/A               339 non-null    float64
 16  AY/A              339 non-null    float64
 1

,Unnamed: 0,Player,Pos,School,Ht,Wt,40yd,Vertical,Broad Jump,3Cone,Shuttle,Year,team,round,pick number,Y/A,AY/A,Rate,Total TDs/season,Rush Avg.
0,0,Tom Brady,QB,Michigan,76,211,5.28,24.5,99.0,7.20,4.38,2000,New England Patriots,6,199,7.5,7.7,138.0,17.0,-0.9
1,1,Travis Brown,QB,Northern Arizona,75,218,5.01,29.0,101.0,7.54,4.87,2000,Undrafted,8,260,NaN,NaN,NaN,NaN,NaN
2,2,Marc Bulger,QB,West Virginia,74,208,4.97,NaN,100.0,7.46,4.34,2000,New Orleans Saints,6,168,NaN,NaN,NaN,NaN,NaN
3,3,Bill Burke,QB,Michigan State,76,206,5.03,28.5,107.0,7.46,4.42,2000,Undrafted,8,260,7.1,5.8,124.7,20.0,-4.1
4,4,Giovanni Carmazzi,QB,Hofstra,75,224,4.74,36.5,119.0,6.94,4.23,2000,San Francisco 49ers,3,65,NaN,NaN,NaN,NaN,NaN


Unnamed: 0          0.00
Player              0.00
Pos                 0.00
School              0.00
Ht                  0.00
Wt                  0.00
40yd                0.04
Vertical            0.18
Broad Jump          0.17
3Cone               0.23
Shuttle             0.22
Year                0.00
team                0.00
round               0.00
pick number         0.00
Y/A                 0.15
AY/A                0.15
Rate                0.15
Total TDs/season    0.15
Rush Avg.           0.15
dtype: float64

In [26]:
print(rb_df.info())
display(rb_df.head())
round(rb_df.isna().sum() / len(rb_df), 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635 entries, 0 to 634
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    635 non-null    int64  
 1   Player                        635 non-null    object 
 2   Pos                           635 non-null    object 
 3   School                        635 non-null    object 
 4   Ht                            635 non-null    int64  
 5   Wt                            635 non-null    int64  
 6   40yd                          619 non-null    float64
 7   Vertical                      528 non-null    float64
 8   Bench                         496 non-null    float64
 9   Broad Jump                    520 non-null    float64
 10  3Cone                         373 non-null    float64
 11  Shuttle                       379 non-null    float64
 12  Year                          635 non-null    int64  
 13  team 

,Unnamed: 0,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Year,team,round,pick number,Avg/Rush,Avg/Rec,Total TDs/season,Total Scrimmage Plays/season
0,0,Shaun Alexander,RB,Alabama,72,218,4.58,NaN,NaN,NaN,NaN,NaN,2000,Seattle Seahawks,1,19,4.6,12.9,23.0,327.0
1,1,Keith Brown,RB,UCLA,71,216,4.66,30.0,NaN,117.0,6.64,4.08,2000,Undrafted,8,260,4.3,2.4,5.0,105.0
2,2,Demario Brown,RB,Utah State,72,217,4.72,31.0,NaN,107.0,7.17,4.39,2000,Undrafted,8,260,5.5,10.8,15.0,305.0
3,3,Trung Canidate,RB,Arizona,71,193,4.41,NaN,18.0,NaN,NaN,NaN,2000,St. Louis Rams,1,31,6.3,8.4,12.0,283.0
4,4,Doug Chapman,RB,Marshall,70,215,4.56,38.5,16.0,128.0,6.84,4.20,2000,Minnesota Vikings,3,88,4.2,13.4,13.0,188.0


Unnamed: 0                      0.00
Player                          0.00
Pos                             0.00
School                          0.00
Ht                              0.00
Wt                              0.00
40yd                            0.03
Vertical                        0.17
Bench                           0.22
Broad Jump                      0.18
3Cone                           0.41
Shuttle                         0.40
Year                            0.00
team                            0.00
round                           0.00
pick number                     0.00
Avg/Rush                        0.14
Avg/Rec                         0.14
Total TDs/season                0.14
Total Scrimmage Plays/season    0.14
dtype: float64

# Pre-processing 

##Final data cleaning steps

In [27]:
#establish dictionary for iterating through
dfs = {'Source Table': df,
       'QB Table': qb_df,
       'WR Table': wr_df,
       'RB Table': rb_df}

In [28]:
#removing 'Year' because of irrelevance and 'pick number' because 
#the target 'round' is too closely correlated
for title, data in dfs.items():
  data.drop(columns = ['Unnamed: 0', 'Year', 'pick number', 'Player'], inplace=True)
  print(title, data.columns)

Source Table Index(['Pos', 'School', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump',
       '3Cone', 'Shuttle', 'team', 'round'],
      dtype='object')
QB Table Index(['Pos', 'School', 'Ht', 'Wt', '40yd', 'Vertical', 'Broad Jump', '3Cone',
       'Shuttle', 'team', 'round', 'Y/A', 'AY/A', 'Rate', 'Total TDs/season',
       'Rush Avg.'],
      dtype='object')
WR Table Index(['Pos', 'School', 'Ht', 'Wt', '40yd', 'Vertical', 'Broad Jump', '3Cone',
       'Shuttle', 'team', 'round', 'Avg/Rec', 'Avg/Rush', 'Total TDs/season',
       'Total Scrimmage Plays/season'],
      dtype='object')
RB Table Index(['Pos', 'School', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump',
       '3Cone', 'Shuttle', 'team', 'round', 'Avg/Rush', 'Avg/Rec',
       'Total TDs/season', 'Total Scrimmage Plays/season'],
      dtype='object')


## Data Validation Split

In [29]:
#Review balance of target column 'round'
for title, data in dfs.items():
  print(title, '\n\n')
  print(round(data['round'].value_counts(normalize=True), 2))
  print('\n\n')

Source Table 


8    0.36
3    0.10
4    0.10
1    0.09
2    0.09
5    0.09
6    0.08
7    0.08
Name: round, dtype: float64



QB Table 


8    0.40
1    0.16
6    0.09
5    0.07
4    0.07
7    0.07
3    0.07
2    0.06
Name: round, dtype: float64



WR Table 


8    0.40
2    0.10
3    0.10
4    0.09
6    0.09
1    0.08
5    0.08
7    0.07
Name: round, dtype: float64



RB Table 


8    0.42
4    0.11
3    0.09
2    0.08
1    0.08
6    0.08
7    0.08
5    0.06
Name: round, dtype: float64





After review, I will prioritize balancing the target, since 30-40% of the values are '8' (meaning 'undrafted').

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7327 entries, 0 to 7326
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pos         7327 non-null   object 
 1   School      7327 non-null   object 
 2   Ht          7327 non-null   int64  
 3   Wt          7327 non-null   int64  
 4   40yd        6956 non-null   float64
 5   Vertical    5696 non-null   float64
 6   Bench       5009 non-null   float64
 7   Broad Jump  5632 non-null   float64
 8   3Cone       4692 non-null   float64
 9   Shuttle     4787 non-null   float64
 10  team        7327 non-null   object 
 11  round       7327 non-null   int64  
dtypes: float64(6), int64(3), object(3)
memory usage: 687.0+ KB


In [31]:
#Establish 'round' as target, split data for each DF
#Resample using SMOTE to balance classes

df_y = df['round']
df_X = df.drop(columns='round')
df_Xtrain, df_Xtest, df_ytrain, df_ytest = train_test_split(df_X, df_y, random_state=42)

In [32]:
qb_y = qb_df['round']
qb_X = qb_df.drop(columns='round')
qb_Xtrain, qb_Xtest, qb_ytrain, qb_ytest = train_test_split(qb_X, qb_y, random_state=42)

In [33]:
wr_y = wr_df['round']
wr_X = wr_df.drop(columns='round')
wr_Xtrain, wr_Xtest, wr_ytrain, wr_ytest = train_test_split(wr_X, wr_y, random_state=42)

In [34]:
rb_y = rb_df['round']
rb_X = rb_df.drop(columns='round')
rb_Xtrain, rb_Xtest, rb_ytrain, rb_ytest = train_test_split(rb_X, rb_y, random_state=42)

##Instantiate Selectors, Imputers, and Transformers

In [36]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
scaler = StandardScaler()

num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

mean_imputer = SimpleImputer()


## Build Pipelines

In [41]:
#Pipelines should impute mean values to nulls, and OneHotEncode object categories
num_pipe = make_pipeline(mean_imputer, scaler)

num_tuple = (num_pipe, num_selector)
cat_tuple = (ohe, cat_selector)

transformer = make_column_transformer(num_tuple, cat_tuple, remainder='passthrough')

##Tuning Hyperparameters and Testing Models

In [42]:
#Instantiate models for tuning
rf = RandomForestRegressor()
lr = LinearRegression()
lgbm = LGBMRegressor()

###Tuning Hyperparameters

In [45]:
#Instantiate model pipelines
rf_pipe = make_pipeline(transformer, rf)
lgbm_pipe = make_pipeline(transformer, lgbm)
lr_pipe = make_pipeline(transformer, lr);

In [67]:
#Instantiate parameter grids for GridCV tuning
rf_params = {'randomforestregressor__max_depth': range(10, 39, 4),
             'randomforestregressor__min_samples_leaf': [1,2,3]}

lgbm_params = {'lgbmregressor__max_depth': range(10, 39, 4),
             'lgbmregressor__num_leaves': range(10, 31, 5),
             'lgbmregressor__class_weight': ['balanced', None]}

#Instantiate dictionary for looping
model_params = {rf_pipe: rf_params,
                lgbm_pipe: lgbm_params}

best_params = []

In [97]:
def grid_tuner(Xtrain=df_Xtrain, ytrain=df_ytrain, Xtest=df_Xtest, 
               ytest = df_ytest):
  results = []
  for model, params in model_params.items():
    grid = GridSearchCV(model, params, verbose=1, cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error')
    grid.fit(Xtrain, ytrain)
    print(grid.best_params_)
    
    ypred_test = grid.predict(Xtest)
    ypred_train = grid.predict(Xtrain)
    r2in_metrics = r2_score(ytrain, ypred_train)
    r2st_metrics = r2_score(ytest, ypred_test)
    rmse_train_metrics = np.sqrt(mean_squared_error(ytrain, ypred_train))
    rmse_test_metrics = np.sqrt(mean_squared_error(ytest, ypred_test))
    
    print('\n\n')
    print(f'Training Set Result (R2):{r2in_metrics}')
    print(f'Testing Set Result (R2):{r2st_metrics}')
    print('\n\n')
    print(f'Training Set Result (RMSE):{rmse_train_metrics}')
    print(f'Test Set Result (RMSE): {rmse_test_metrics}')
    
    best_params.append(grid.best_params_)
    results.append(grid.cv_results_)

In [83]:
#Run metrics for each DF and then view reports
grid_tuner()

Fitting 3 folds for each of 24 candidates, totalling 72 fits
{'randomforestregressor__max_depth': 14, 'randomforestregressor__min_samples_leaf': 3}



Training Set Result (R2):0.8331385783636008
Testing Set Result (R2):0.6590165174614859



Training Set Result (RMSE):1.0349489998998196
Test Set Result (RMSE): 1.4477118581182058
Fitting 3 folds for each of 80 candidates, totalling 240 fits
{'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}



Training Set Result (R2):0.7197484842507436
Testing Set Result (R2):0.6594789614395371



Training Set Result (RMSE):1.3412661509526704
Test Set Result (RMSE): 1.446729827009279


In [84]:
print(best_params)
pd.DataFrame(results)

[{'randomforestregressor__max_depth': 14, 'randomforestregressor__min_samples_leaf': 2}, {'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}, {'randomforestregressor__max_depth': 10, 'randomforestregressor__min_samples_leaf': 1}, {'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}, {'randomforestregressor__max_depth': 14, 'randomforestregressor__min_samples_leaf': 3}, {'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}]


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,param_lgbmregressor__class_weight,param_lgbmregressor__max_depth,param_lgbmregressor__num_leaves
0,"[2.94917885462443, 2.94029966990153, 2.8360768...","[0.013157816385388786, 0.022034481728350548, 0...","[0.041016578674316406, 0.04275051752726237, 0....","[0.0011584141392210184, 0.0036719032571869378,...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.4679055972453707, -1.4625644752538236, -1....","[-1.4491148729943455, -1.441818762083525, -1.4...","[-1.5351929136958038, -1.5317411086195534, -1....","[-1.4840711279785068, -1.4787081153189672, -1....","[0.03695357953490162, 0.03844450491770424, 0.0...","[14, 2, 5, 15, 1, 4, 18, 3, 7, 17, 8, 12, 24, ...",NaN,NaN,NaN
1,"[0.11924552917480469, 0.1372969150543213, 0.15...","[0.005581827555082766, 0.006192543918033426, 0...","[0.02211475372314453, 0.027941385904947918, 0....","[0.00032059530879591, 0.004770467187752463, 0....",NaN,NaN,"[{'lgbmregressor__class_weight': 'balanced', '...","[-1.492349083760785, -1.4853152340800029, -1.4...","[-1.467810028866227, -1.4703741752968191, -1.4...","[-1.5258226023408226, -1.5343465647552765, -1....","[-1.4953272383226113, -1.4966786580440328, -1....","[0.023776973787426256, 0.02732474100390652, 0....","[28, 36, 64, 65, 72, 28, 45, 63, 66, 80, 28, 4...","[balanced, balanced, balanced, balanced, balan...","[10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 18, 1...","[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 1..."
2,"[0.27106237411499023, 0.24889016151428223, 0.2...","[0.01616228342591879, 0.008497822715027028, 0....","[0.01961064338684082, 0.019109010696411133, 0....","[0.001344615388524467, 0.0011555213164339934, ...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.6538043555317021, -1.6566381881159555, -1....","[-1.4571969603894948, -1.4518254253087917, -1....","[-1.7323832896233808, -1.7633011198702813, -1....","[-1.614461535181526, -1.6239215777650093, -1.6...","[0.11573755392880586, 0.12924668795313432, 0.1...","[1, 3, 15, 6, 4, 24, 5, 20, 13, 8, 19, 9, 23, ...",NaN,NaN,NaN
3,"[0.039244492848714195, 0.04181059201558431, 0....","[0.0001710963368791064, 0.0021871347496175337,...","[0.00977476437886556, 0.008078654607137045, 0....","[0.0027648361840377253, 0.0006113873801153849,...",NaN,NaN,"[{'lgbmregressor__class_weight': 'balanced', '...","[-1.7446966693327919, -1.7446966693327919, -1....","[-1.5969015768441917, -1.5969015768441917, -1....","[-2.053778199970844, -2.053778199970844, -2.05...","[-1.7984588153826093, -1.7984588153826093, -1....","[0.19035377318879138, 0.19035377318879138, 0.1...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[balanced, balanced, balanced, balanced, balan...","[10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 18, 1...","[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 1..."
4,"[3.5092904567718506, 3.1050643920898438, 2.927...","[0.10828542107103697, 0.17830553769281857, 0.0...","[0.04338685671488444, 0.044345458348592125, 0....","[0.0033733582102628027, 0.0008683507596775477,...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.4673342778626532, -1.4642815646105485, -1....","[-1.4501590816899061, -1.4468101653378838, -1....","[-1.533454589723929, -1.5328162432291124, -1.5...","[-1.4836493164254962, -1.4813026577258483, -1....","[0.03590887334296612, 0.03711737748178336, 0.0...","[10, 7, 3, 12, 6, 1, 17, 13, 2, 19, 15, 4, 23,...",NaN,NaN,NaN
5,"[0.11704071362813313, 0.14166688919067383, 0.1...","[0.004473446917968503, 0.006603671100788365, 0..."

In [90]:
#Test LR model 
def lr_tester(Xtrain=df_Xtrain, ytrain=df_ytrain, Xtest=df_Xtest, 
               ytest = df_ytest):
  lr_pipe.fit(Xtrain, ytrain)

  ypred_test = lr_pipe.predict(Xtest)
  ypred_train = lr_pipe.predict(Xtrain)
  r2in_metrics = r2_score(ytrain, ypred_train)
  r2st_metrics = r2_score(ytest, ypred_test)
  rmse_train_metrics = np.sqrt(mean_squared_error(ytrain, ypred_train))
  rmse_test_metrics = np.sqrt(mean_squared_error(ytest, ypred_test))

  print('\n\n')
  print(f'Training Set Result (R2):{r2in_metrics}')
  print(f'Testing Set Result (R2):{r2st_metrics}')
  print('\n\n')
  print(f'Training Set Result (RMSE):{rmse_train_metrics}')
  print(f'Test Set Result (RMSE): {rmse_test_metrics}')

In [91]:
lr_tester()




Training Set Result (R2):0.671996694912907
Testing Set Result (R2):-2.391123311808963e+17



Training Set Result (RMSE):1.451042355112065
Test Set Result (RMSE): 1212317704.1984832


In [77]:
grid_tuner(Xtrain=qb_Xtrain, ytrain=qb_ytrain, Xtest=qb_Xtest, 
               ytest = qb_ytest)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
{'randomforestregressor__max_depth': 10, 'randomforestregressor__min_samples_leaf': 1}



Training Set Result (R2):0.9411710414211282
Testing Set Result (R2):0.7071351116092678



Training Set Result (RMSE):0.663715625885545
Test Set Result (RMSE): 1.399258968196923
Fitting 3 folds for each of 80 candidates, totalling 240 fits
{'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}



Training Set Result (R2):0.9430026062669885
Testing Set Result (R2):0.691109258724071



Training Set Result (RMSE):0.6533019597559103
Test Set Result (RMSE): 1.4370334949181578


In [120]:
pd.DataFrame(results)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,param_lgbmregressor__class_weight,param_lgbmregressor__max_depth,param_lgbmregressor__num_leaves
0,"[2.94917885462443, 2.94029966990153, 2.8360768...","[0.013157816385388786, 0.022034481728350548, 0...","[0.041016578674316406, 0.04275051752726237, 0....","[0.0011584141392210184, 0.0036719032571869378,...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.4679055972453707, -1.4625644752538236, -1....","[-1.4491148729943455, -1.441818762083525, -1.4...","[-1.5351929136958038, -1.5317411086195534, -1....","[-1.4840711279785068, -1.4787081153189672, -1....","[0.03695357953490162, 0.03844450491770424, 0.0...","[14, 2, 5, 15, 1, 4, 18, 3, 7, 17, 8, 12, 24, ...",NaN,NaN,NaN
1,"[0.11924552917480469, 0.1372969150543213, 0.15...","[0.005581827555082766, 0.006192543918033426, 0...","[0.02211475372314453, 0.027941385904947918, 0....","[0.00032059530879591, 0.004770467187752463, 0....",NaN,NaN,"[{'lgbmregressor__class_weight': 'balanced', '...","[-1.492349083760785, -1.4853152340800029, -1.4...","[-1.467810028866227, -1.4703741752968191, -1.4...","[-1.5258226023408226, -1.5343465647552765, -1....","[-1.4953272383226113, -1.4966786580440328, -1....","[0.023776973787426256, 0.02732474100390652, 0....","[28, 36, 64, 65, 72, 28, 45, 63, 66, 80, 28, 4...","[balanced, balanced, balanced, balanced, balan...","[10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 18, 1...","[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 1..."
2,"[0.27106237411499023, 0.24889016151428223, 0.2...","[0.01616228342591879, 0.008497822715027028, 0....","[0.01961064338684082, 0.019109010696411133, 0....","[0.001344615388524467, 0.0011555213164339934, ...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.6538043555317021, -1.6566381881159555, -1....","[-1.4571969603894948, -1.4518254253087917, -1....","[-1.7323832896233808, -1.7633011198702813, -1....","[-1.614461535181526, -1.6239215777650093, -1.6...","[0.11573755392880586, 0.12924668795313432, 0.1...","[1, 3, 15, 6, 4, 24, 5, 20, 13, 8, 19, 9, 23, ...",NaN,NaN,NaN
3,"[0.039244492848714195, 0.04181059201558431, 0....","[0.0001710963368791064, 0.0021871347496175337,...","[0.00977476437886556, 0.008078654607137045, 0....","[0.0027648361840377253, 0.0006113873801153849,...",NaN,NaN,"[{'lgbmregressor__class_weight': 'balanced', '...","[-1.7446966693327919, -1.7446966693327919, -1....","[-1.5969015768441917, -1.5969015768441917, -1....","[-2.053778199970844, -2.053778199970844, -2.05...","[-1.7984588153826093, -1.7984588153826093, -1....","[0.19035377318879138, 0.19035377318879138, 0.1...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[balanced, balanced, balanced, balanced, balan...","[10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 18, 1...","[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 1..."
4,"[3.5092904567718506, 3.1050643920898438, 2.927...","[0.10828542107103697, 0.17830553769281857, 0.0...","[0.04338685671488444, 0.044345458348592125, 0....","[0.0033733582102628027, 0.0008683507596775477,...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.4673342778626532, -1.4642815646105485, -1....","[-1.4501590816899061, -1.4468101653378838, -1....","[-1.533454589723929, -1.5328162432291124, -1.5...","[-1.4836493164254962, -1.4813026577258483, -1....","[0.03590887334296612, 0.03711737748178336, 0.0...","[10, 7, 3, 12, 6, 1, 17, 13, 2, 19, 15, 4, 23,...",NaN,NaN,NaN
5,"[0.11704071362813313, 0.14166688919067383, 0.1...","[0.004473446917968503, 0.006603671100788365, 0..."

In [85]:
lr_tester(Xtrain=qb_Xtrain, ytrain=qb_ytrain, Xtest=qb_Xtest, 
               ytest = qb_ytest)




Training Set Result (R2):0.8553418754574886
Testing Set Result (R2):-3.217412845490575e+20



Training Set Result (RMSE):1.040777457966769
Test Set Result (RMSE): 46378622047.75375


In [98]:
grid_tuner(Xtrain=rb_Xtrain, ytrain=rb_ytrain, Xtest=rb_Xtest, 
               ytest = rb_ytest)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
{'randomforestregressor__max_depth': 38, 'randomforestregressor__min_samples_leaf': 2}



Training Set Result (R2):0.9495531715258743
Testing Set Result (R2):0.6494193183604331



Training Set Result (RMSE):0.5693576371477812
Test Set Result (RMSE): 1.410474720202622
Fitting 3 folds for each of 80 candidates, totalling 240 fits
{'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}



Training Set Result (R2):0.9462677189576968
Testing Set Result (R2):0.6272670326521008



Training Set Result (RMSE):0.587605504333674
Test Set Result (RMSE): 1.4543543009062636


In [ ]:
display(pd.DataFrame(best_params))
pd.DataFrame(results)

In [101]:
lr_tester(Xtrain=rb_Xtrain, ytrain=rb_ytrain, Xtest=rb_Xtest, 
               ytest = rb_ytest)




Training Set Result (R2):0.8355527900354466
Testing Set Result (R2):-9.379371864794329e+21



Training Set Result (RMSE):1.0279724399205283
Test Set Result (RMSE): 230705630340.9493


In [102]:
grid_tuner(Xtrain=wr_Xtrain, ytrain=wr_ytrain, Xtest=wr_Xtest, 
               ytest = wr_ytest)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
{'randomforestregressor__max_depth': 14, 'randomforestregressor__min_samples_leaf': 2}



Training Set Result (R2):0.9319758617858089
Testing Set Result (R2):0.718403219688889



Training Set Result (RMSE):0.6559793334474096
Test Set Result (RMSE): 1.3562742393254699
Fitting 3 folds for each of 80 candidates, totalling 240 fits
{'lgbmregressor__class_weight': None, 'lgbmregressor__max_depth': 10, 'lgbmregressor__num_leaves': 10}



Training Set Result (R2):0.8898911035974817
Testing Set Result (R2):0.6935581682812287



Training Set Result (RMSE):0.8345835836904998
Test Set Result (RMSE): 1.4148411915137442


In [103]:
display(pd.DataFrame(best_params))
pd.DataFrame(results)

,randomforestregressor__max_depth,randomforestregressor__min_samples_leaf,lgbmregressor__class_weight,lgbmregressor__max_depth,lgbmregressor__num_leaves
0,14.0,2.0,NaN,NaN,NaN
1,NaN,NaN,NaN,10.0,10.0
2,10.0,1.0,NaN,NaN,NaN
3,NaN,NaN,NaN,10.0,10.0
4,14.0,3.0,NaN,NaN,NaN
5,NaN,NaN,NaN,10.0,10.0
6,38.0,2.0,NaN,NaN,NaN
7,NaN,NaN,NaN,10.0,10.0
8,14.0,2.0,NaN,NaN,NaN
9,NaN,NaN,NaN,10.0,10.0


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,param_lgbmregressor__class_weight,param_lgbmregressor__max_depth,param_lgbmregressor__num_leaves
0,"[2.94917885462443, 2.94029966990153, 2.8360768...","[0.013157816385388786, 0.022034481728350548, 0...","[0.041016578674316406, 0.04275051752726237, 0....","[0.0011584141392210184, 0.0036719032571869378,...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.4679055972453707, -1.4625644752538236, -1....","[-1.4491148729943455, -1.441818762083525, -1.4...","[-1.5351929136958038, -1.5317411086195534, -1....","[-1.4840711279785068, -1.4787081153189672, -1....","[0.03695357953490162, 0.03844450491770424, 0.0...","[14, 2, 5, 15, 1, 4, 18, 3, 7, 17, 8, 12, 24, ...",NaN,NaN,NaN
1,"[0.11924552917480469, 0.1372969150543213, 0.15...","[0.005581827555082766, 0.006192543918033426, 0...","[0.02211475372314453, 0.027941385904947918, 0....","[0.00032059530879591, 0.004770467187752463, 0....",NaN,NaN,"[{'lgbmregressor__class_weight': 'balanced', '...","[-1.492349083760785, -1.4853152340800029, -1.4...","[-1.467810028866227, -1.4703741752968191, -1.4...","[-1.5258226023408226, -1.5343465647552765, -1....","[-1.4953272383226113, -1.4966786580440328, -1....","[0.023776973787426256, 0.02732474100390652, 0....","[28, 36, 64, 65, 72, 28, 45, 63, 66, 80, 28, 4...","[balanced, balanced, balanced, balanced, balan...","[10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 18, 1...","[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 1..."
2,"[0.27106237411499023, 0.24889016151428223, 0.2...","[0.01616228342591879, 0.008497822715027028, 0....","[0.01961064338684082, 0.019109010696411133, 0....","[0.001344615388524467, 0.0011555213164339934, ...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.6538043555317021, -1.6566381881159555, -1....","[-1.4571969603894948, -1.4518254253087917, -1....","[-1.7323832896233808, -1.7633011198702813, -1....","[-1.614461535181526, -1.6239215777650093, -1.6...","[0.11573755392880586, 0.12924668795313432, 0.1...","[1, 3, 15, 6, 4, 24, 5, 20, 13, 8, 19, 9, 23, ...",NaN,NaN,NaN
3,"[0.039244492848714195, 0.04181059201558431, 0....","[0.0001710963368791064, 0.0021871347496175337,...","[0.00977476437886556, 0.008078654607137045, 0....","[0.0027648361840377253, 0.0006113873801153849,...",NaN,NaN,"[{'lgbmregressor__class_weight': 'balanced', '...","[-1.7446966693327919, -1.7446966693327919, -1....","[-1.5969015768441917, -1.5969015768441917, -1....","[-2.053778199970844, -2.053778199970844, -2.05...","[-1.7984588153826093, -1.7984588153826093, -1....","[0.19035377318879138, 0.19035377318879138, 0.1...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[balanced, balanced, balanced, balanced, balan...","[10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 18, 1...","[10, 15, 20, 25, 30, 10, 15, 20, 25, 30, 10, 1..."
4,"[3.5092904567718506, 3.1050643920898438, 2.927...","[0.10828542107103697, 0.17830553769281857, 0.0...","[0.04338685671488444, 0.044345458348592125, 0....","[0.0033733582102628027, 0.0008683507596775477,...","[10, 10, 10, 14, 14, 14, 18, 18, 18, 22, 22, 2...","[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, ...","[{'randomforestregressor__max_depth': 10, 'ran...","[-1.4673342778626532, -1.4642815646105485, -1....","[-1.4501590816899061, -1.4468101653378838, -1....","[-1.533454589723929, -1.5328162432291124, -1.5...","[-1.4836493164254962, -1.4813026577258483, -1....","[0.03590887334296612, 0.03711737748178336, 0.0...","[10, 7, 3, 12, 6, 1, 17, 13, 2, 19, 15, 4, 23,...",NaN,NaN,NaN
5,"[0.11704071362813313, 0.14166688919067383, 0.1...","[0.004473446917968503, 0.006603671100788365, 0..."

In [104]:
lr_tester(Xtrain=wr_Xtrain, ytrain=wr_ytrain, Xtest=wr_Xtest, 
               ytest = wr_ytest)




Training Set Result (R2):0.773064866306165
Testing Set Result (R2):-6.469581414489082e+19



Training Set Result (RMSE):1.1981453368951418
Test Set Result (RMSE): 20557569243.509747


#Instantiating and Testing Final Model


The Random Forest model produced the best r2 and rmse scores for each model. Therefore, I will be using this for each DF.

In [115]:
#Set up function to evaluate final models
def model_eval(train=df_ytrain, test=df_ytest):
  r2in_metrics = r2_score(train, train_preds)
  r2st_metrics = r2_score(test, test_preds)
  rmse_train_metrics = np.sqrt(mean_squared_error(train, train_preds))
  rmse_test_metrics = np.sqrt(mean_squared_error(test, test_preds))

  print('\n\n')
  print(f'Training Set Result (R2):{r2in_metrics}')
  print(f'Testing Set Result (R2):{r2st_metrics}')
  print('\n\n')
  print(f'Training Set Result (RMSE):{rmse_train_metrics}')
  print(f'Test Set Result (RMSE): {rmse_test_metrics}')

##Main Table

In [108]:
best_params[0]

{'randomforestregressor__max_depth': 14,
 'randomforestregressor__min_samples_leaf': 2}

In [109]:
#Instantiate tuned main table model and then run metrics
rf = RandomForestRegressor(max_depth=14, min_samples_leaf=2)
rf_pipe = make_pipeline(transformer, rf)

In [111]:
rf_pipe.fit(df_Xtrain, df_ytrain)
train_preds = rf_pipe.predict(df_Xtrain)
test_preds = rf_pipe.predict(df_Xtest)

In [112]:
model_eval()




Training Set Result (R2):0.8414831060833765
Testing Set Result (R2):0.6574391742862933



Training Set Result (RMSE):1.0087388712154097
Test Set Result (RMSE): 1.4510564536658601


##QB Table

In [113]:
best_params[2]

{'randomforestregressor__max_depth': 10,
 'randomforestregressor__min_samples_leaf': 1}

In [122]:
#Instantiate tuned QB table modle and then run metrics
rf_qb = RandomForestRegressor(max_depth=10, min_samples_leaf=1)
rf_qb_pipe = make_pipeline(transformer, rf_qb)

In [123]:
rf_qb_pipe.fit(qb_Xtrain, qb_ytrain)
train_preds = rf_qb_pipe.predict(qb_Xtrain)
test_preds = rf_qb_pipe.predict(qb_Xtest);

In [124]:
model_eval(train=qb_ytrain, test=qb_ytest)




Training Set Result (R2):0.9423667686307843
Testing Set Result (R2):0.7118522561965962



Training Set Result (RMSE):0.6569358268159068
Test Set Result (RMSE): 1.3879443624193668


##RB Table

In [119]:
best_params[6]

{'randomforestregressor__max_depth': 38,
 'randomforestregressor__min_samples_leaf': 2}

In [125]:
#Instantiate tuned model and then run metrics
rf_rb = RandomForestRegressor(max_depth=38, min_samples_leaf=2)

In [126]:
rf_rb_pipe = make_pipeline(transformer, rf_rb)

In [127]:
rf_rb_pipe.fit(rb_Xtrain, rb_ytrain)
train_preds = rf_rb_pipe.predict(rb_Xtrain)
test_preds = rf_rb_pipe.predict(rb_Xtest);

In [128]:
model_eval(train=rb_ytrain, test=rb_ytest)




Training Set Result (R2):0.9518994889722213
Testing Set Result (R2):0.6434546352158264



Training Set Result (RMSE):0.5559593797567947
Test Set Result (RMSE): 1.4224228282659215


##WR Table

In [129]:
best_params[8]

{'randomforestregressor__max_depth': 14,
 'randomforestregressor__min_samples_leaf': 2}

In [130]:
#Instantiate tuned model and then run metrics
rf_wr = RandomForestRegressor(max_depth=14, min_samples_leaf=2)
rf_wr_pipe = make_pipeline(transformer, rf_wr)

In [131]:
rf_wr_pipe.fit(wr_Xtrain, wr_ytrain)
train_preds = rf_wr_pipe.predict(wr_Xtrain)
test_preds = rf_wr_pipe.predict(wr_Xtest);

In [132]:
model_eval(train=wr_ytrain, test=wr_ytest)




Training Set Result (R2):0.9302888176542723
Testing Set Result (R2):0.7145866251193509



Training Set Result (RMSE):0.6640638780493723
Test Set Result (RMSE): 1.3654343715306811
